In [1]:
import sys
sys.path.append("../../current_work/")
from pandas.core.indexing import maybe_convert_ix
import PredictionManager as pm
import backtest_class as backTest
import yFinance_class as yfHelper

## Data'nın indirilmesi

In [2]:
# use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        #period = "ytd",

# fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        #interval = "1m",

In [3]:
userInput = "SASA.IS"
period = '1mo'
interval = '15m'

In [4]:
yData = yfHelper.yFinance(userInput.upper(), period, interval).getData()

[*********************100%***********************]  1 of 1 completed


In [5]:
yData.iloc[0]

Date         2021-11-11 10:30:00+03:00
Open                             32.18
High                             32.24
Low                              32.14
Close                            32.18
Adj Close                        32.18
Volume                         1332163
Name: 0, dtype: object

In [6]:
yData.iloc[-1]

Date         2021-12-10 18:00:00+03:00
Open                             47.02
High                             47.02
Low                              47.02
Close                            47.02
Adj Close                        47.02
Volume                          578875
Name: 681, dtype: object

## RSI parametresinin çıkartılması

In [7]:
import pandas as pd
import numpy as np

In [8]:
close = yData.Close
ret = close.diff() #(n+1).gün - n.gün
up = []
down = [] 

for element in range(0, len(ret)):
    if (ret[element] < np.float(0)):
        up.append(0)
        down.append(ret[element])
    elif (ret[element] >= np.float(0)):
        up.append(ret[element])
        down.append(0)
        
upSeries = pd.Series(up)
downSeries = pd.Series(down).abs()

In [9]:
ret = ret.replace(np.nan, 0)
ret

0      0.000000
1      0.239998
2     -0.099998
3      0.139999
4      0.139999
         ...   
677   -0.079998
678    0.000000
679    0.059998
680    0.220001
681   -0.320000
Name: Close, Length: 682, dtype: float64

In [10]:
lookback = 7
upEwm = upSeries.ewm(com = lookback -1, adjust = False).mean()
downEwm = downSeries.ewm(com = lookback -1, adjust = False).mean()

In [11]:
rs = upEwm / downEwm
rsi = 100 - (100 / (1 + rs))

In [12]:
close = close.drop(index = close.index[0])
temp_df = pd.DataFrame(rsi).rename(columns={0 : 'rsi'}).set_index(close.index)


In [13]:
print("toplam gün sayısı:", len(yData))
print("rsi veri gün sayısı:", len(rsi))

toplam gün sayısı: 682
rsi veri gün sayısı: 681


In [14]:
#bu işlemi yapmamızın sebebi hesaplamalarda ilk iki gün 
#RSI değerinin 100 çıkması
#o yüzden ilk 2 günü pass geçiyoruz hatta 3 günü de geçebiliriz.
yData["RSI_14"] = temp_df[2:]

## Algoritmanın geliştirilmesi

### 1- RSI ile alım satım

In [15]:
lower_band = np.float(30)
upper_band = np.float(70)
signal = 0 #sinyal parametresi sattıysak -1 aldıysak 1 olacak.
rsi_signal = []
buy_price = [] #alım yaptıgımızda alım fiyatını ekledigimiz liste
sell_price = []#satış yaptıgımızda satış fiyatını ekledigimiz liste
rsi = yData.RSI_14
prices = yData.Close
def wait_for_signal(buyList, sellList, signalList):
    buyList.append(np.nan)
    sellList.append(np.nan)
    signalList.append(0)
    
def fill_buy_signal(buyList,buyPrice, sellList, signalList):
    buyList.append(buyPrice)
    sellList.append(np.nan)
    signalList.append(1)
    
def fill_sell_signal(buyList, sellList, sellPrice, signalList):
    buyList.append(np.nan)
    sellList.append(sellPrice)
    signalList.append(-1)
    
for element in range(1, len(rsi)):
    if ((rsi[element - 1] > lower_band) and (rsi[element] < lower_band)):
        #eğer rsi değeri 30'un üstünden 30'un altına geçiş yaptıysa
        #alım yapmak istiyoruz
        #ancak iki kere alım yapamayacağımızdan sinyal kontrolü yap.
        if (signal != 1):
            fill_buy_signal(buy_price, prices[element], sell_price, rsi_signal)
            signal = 1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
    elif((rsi[element - 1] < upper_band) and (rsi[element] > upper_band)):
        #eğer rsi değeri 70'in altındaysa ve 70'in üstüne geçiş yaptıysa
        #satış yapmak istiyoruz
        #ancak iki kere satış yapamacağımızdan sinyal kontrolü yap
        if (signal != -1):
            fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
            signal = -1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
    else:
        wait_for_signal(buy_price, sell_price, rsi_signal)
            
            
        

In [16]:
investmenValue = yData['Close'][0] * 5
firstInvestment = investmenValue
decisionFlag = False
stockNumber = round(investmenValue / yData['Close'][0])


for i in range(0, len(rsi) - 1):
    if (rsi_signal[i] != 0):
        if (decisionFlag == False):
            if(rsi_signal[i] == 1):
                decisionFlag = True
            elif(rsi_signal[i] == -1):
                decisionFlag = True
                investmenValue = yData["Close"][i+1] * stockNumber    
        else:
            if (rsi_signal[i] == 1):
                stockNumber = round(investmenValue / yData['Close'][i+1])
                investmenValue = 0
            elif (rsi_signal[i] == -1):
                investmenValue = yData['Close'][i+1] * stockNumber

                stockNumber = 0
      


if (investmenValue == 0.0):
    investmenValue = yData['Close'][len(yData) - 1] * stockNumber
    stockNumber = 0
    
print("% ",(investmenValue - firstInvestment) / firstInvestment * 100)
print(investmenValue - firstInvestment)

%  46.1155997882357
74.20000076293945


In [17]:
print("işlem yapılmasa durum: %",(yData.Close.iloc[-1] - yData.Close.iloc[0]) / yData.Close.iloc[0]*100)

işlem yapılmasa durum: % 46.1155997882357


In [18]:
alim_satim = 0
for i in rsi_signal:
    alim_satim += abs(i)
    
print("alım satım sayısı: ", alim_satim)
print("veri uzunlugu: ", len(yData))

alım satım sayısı:  19
veri uzunlugu:  682


### 2- Safe Sell RSI algoritması

In [19]:
lower_band = np.float(30)
upper_band = np.float(70)
signal = 0 #sinyal parametresi sattıysak -1 aldıysak 1 olacak.
rsi_signal = []
buy_price = [] #alım yaptıgımızda alım fiyatını ekledigimiz liste
sell_price = []#satış yaptıgımızda satış fiyatını ekledigimiz liste
rsi = yData.RSI_14
beklenenKar = 0.01
prices = yData.Close
lastBuyPrice = []
def wait_for_signal(buyList, sellList, signalList):
    buyList.append(np.nan)
    sellList.append(np.nan)
    signalList.append(0)
    
def fill_buy_signal(buyList,buyPrice, sellList, signalList):
    buyList.append(buyPrice)
    sellList.append(np.nan)
    signalList.append(1)
    
def fill_sell_signal(buyList, sellList, sellPrice, signalList):
    buyList.append(np.nan)
    sellList.append(sellPrice)
    signalList.append(-1)
    
for element in range(1, len(rsi)):
    if ((rsi[element - 1] > lower_band) and (rsi[element] < lower_band)):
        #eğer rsi değeri 30'un üstünden 30'un altına geçiş yaptıysa
        #alım yapmak istiyoruz
        #ancak iki kere alım yapamayacağımızdan sinyal kontrolü yap.
        if (signal != 1):
            fill_buy_signal(buy_price, prices[element], sell_price, rsi_signal)
            signal = 1
            lastBuyPrice.append(prices[element])
        else:
            #RSI alım sinyali verdi ama zaten alım yapmışız
            #fakat karımız %5 üzerindeyse satış yapmak istiyoruz.
            #o yüzden ilk önce fiyat düşüşte mi ona bakıyoruz
            if (prices[element - 1] > prices[element]):
                #eğer düşüşteyse aldığımız fiyattan yüksek mi
                #onu kontrol etmeliyiz.
                if (prices[element] > lastBuyPrice[-1]):
                    #eğer öyleyse karımızı hesaplamalıyız
                    lossPortion = (prices[element] - lastBuyPrice[-1]) / lastBuyPrice[-1]
                    if (lossPortion > beklenenKar ):
                        print(lossPortion)
                        fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
                        signal = -1
                    else:
                        wait_for_signal(buy_price, sell_price, rsi_signal)
                else:
                    wait_for_signal(buy_price, sell_price, rsi_signal)
            else:
                wait_for_signal(buy_price, sell_price, rsi_signal)
            
    elif((rsi[element - 1] < upper_band) and (rsi[element] > upper_band)):
        #eğer rsi değeri 70'in altındaysa ve 70'in üstüne geçiş yaptıysa
        #satış yapmak istiyoruz
        #ancak iki kere satış yapamacağımızdan sinyal kontrolü yap
        if (signal != -1):
            fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
            signal = -1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
         #satış pozisyonunda bir daha satış yapamayacağımızdan burayı pass geçiyoruz.   
    else:
        #eğer bu durumlardan biri yaşanmadıysa, karı kontrol edebiliriz.
        #ilk olarak sinyal alım pozisyonunda mı ona bakıyoruz.
        if (signal == 1):
            if (prices[element - 1] > prices[element]):
                #fiyat düşüştüyse tespit ettik.
                if (prices[element] > lastBuyPrice[-1]):
                    #karda olup olmadığımızı kontrol ettik
                    lossPortion = (prices[element] - lastBuyPrice[-1]) / lastBuyPrice[-1]
                    if (lossPortion > beklenenKar):
                        print(lossPortion)
                        fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
                        signal = -1
                    else:
                        wait_for_signal(buy_price, sell_price, rsi_signal) 
                else:
                    wait_for_signal(buy_price, sell_price, rsi_signal)
            else:
                wait_for_signal(buy_price, sell_price, rsi_signal)
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
        

0.018419457358744853
0.01502595476638536
0.027239416113528907
0.018227879005142406
0.011685403545251053


In [20]:
alim_satim = 0
for i in rsi_signal:
    alim_satim += abs(i)
    
print("alım satım sayısı: ", alim_satim)
print("veri uzunlugu: ", len(yData))

alım satım sayısı:  21
veri uzunlugu:  682


In [21]:
investmenValue = yData['Close'][0] * 5
firstInvestment = investmenValue
decisionFlag = False
stockNumber = round(investmenValue / yData['Close'][0])



for i in range(0, len(rsi) - 1):
    if (rsi_signal[i] != 0):
        if (decisionFlag == False):
            if(rsi_signal[i] == 1):
                decisionFlag = True
            elif(rsi_signal[i] == -1):
                decisionFlag = True
                investmenValue = yData["Close"][i+1] * stockNumber    
        else:
            if (rsi_signal[i] == 1):
                stockNumber = round(investmenValue / yData['Close'][i+1])
                investmenValue = 0
            elif (rsi_signal[i] == -1):
                investmenValue = yData['Close'][i+1] * stockNumber
                stockNumber = 0
                    
if (investmenValue == 0):
    investmenValue = yData['Close'][len(yData) - 1] * stockNumber
    stockNumber = 0

In [22]:
print("% ",(investmenValue - firstInvestment) / firstInvestment * 100)
print(investmenValue - firstInvestment)

%  16.89247983058856
27.18000030517578


In [23]:
print("işlem yapılmasa durum: %",(yData.Close.iloc[-1] - yData.Close.iloc[0]) / yData.Close.iloc[0] * 100)

işlem yapılmasa durum: % 46.1155997882357
